In [502]:
import numpy as np
import matplotlib.pyplot as plt

import math

import sympy as sp

from sympy import latex
from IPython.display import display, Math

## Things which are given / can be calculated 

In [503]:
P_d_sp = sp.symbols('P_d')
D_p_sp, D_g_sp = sp.symbols('D_p D_g')
N_p_sp, N_g_sp, N_g_guess_sp = sp.symbols('N_p N_g, N_g_guess') 
C_sp, F_sp = sp.symbols('C F')
n_p_sp, n_g_sp, n_g_guess_sp = sp.symbols('n_p n_g n_g_guess')
VR_sp, VR_guess_sp, v_t_sp = sp.symbols('VR VR_guess v_t')
power_sp, lifetime_sp, Torque_sp = sp.symbols('power lifetime torque')
W_t_sp = sp.symbols('W_t')


#enter given values

uk = sp.nan
all_values = {
    P_d_sp: 10,
    D_p_sp: sp.nan,
    D_g_sp: uk,
    
    N_p_sp: 20,
    N_g_sp: uk,
    N_g_guess_sp: uk,
    n_p_sp: 1750,
    n_g_sp: uk,
    n_g_guess_sp: 580,
    VR_sp: sp.nan,
    VR_guess_sp: sp.nan,
    v_t_sp: sp.nan, 
    Torque_sp: uk,
    power_sp: 10
}

phi = 20 # pressure angle

lifetime = 1500
F_p = 1 # update if given
F_g = 1 # update if given

all_values, len(all_values)


({P_d: 10,
  D_p: nan,
  D_g: nan,
  N_p: 20,
  N_g: nan,
  N_g_guess: nan,
  n_p: 1750,
  n_g: nan,
  n_g_guess: 580,
  VR: nan,
  VR_guess: nan,
  v_t: nan,
  torque: nan,
  power: 10},
 14)

## Things to find from a table

some, like Y,Z can only be found later, tbd how to handle this

In [504]:
J_p = 0.2475
J_G = uk

I = 0.0825

C_p = 2300 #for steel

k_o = 1.5
k_s = 0.50803

k_B = uk
k_v = 1.14

#for 99% reliability
k_R = 1.00
C_R = 1.00


#Need cycles to find, but these are not calculated until later
#im not sure the best way to handle this
Y_np = uk
Y_ng = uk

Z_np = uk
Z_ng = uk


#for k_mb
C_ma_cond = "commercial enclosed"
# C_ma_cond = "precicision"
# C_ma_cond = "extra"

k_ma_cond = "2 straddle"
# k_ma_cond = "1 straddle"
# k_ma_cond = "0 straddle"



#for C_xc
teeth = "crowned"



safety_factor = 1


## Create the equations

In [505]:

eq1 = sp.Eq(VR_guess_sp, n_p_sp / n_g_guess_sp)
eq2 = sp.Eq(N_g_guess_sp, N_p_sp * VR_guess_sp)

eq3 = sp.Eq(VR_sp, N_g_sp / N_p_sp)

eq4 = sp.Eq(n_g_sp, n_p_sp / VR_sp)
eq5 = sp.Eq(D_p_sp, N_p_sp / P_d_sp)
eq6 = sp.Eq(D_g_sp, N_g_sp / P_d_sp)

eq7 = sp.Eq(v_t_sp, np.pi * D_p_sp * n_p_sp / 12)
eq8 = sp.Eq(VR_sp, n_p_sp / n_g_sp)

eq9 = sp.Eq(power_sp, Torque_sp * n_p_sp / 63000)


## Solve the equations for basic quantities

In [506]:
#creates a thing of all the non-nan values
known_values = {}

for var, value in all_values.items():
    if value is not sp.nan:
        known_values[var] = value


Solve the equations for VR_guess, n_g_guess, N_p, n_g

In [507]:
# Solve equations eq1 and eq2
solutions_eq1_eq2 = sp.solve([eq1, eq2], dict=True)

# Display the solutions
# for solution in solutions_eq1_eq2:
#     for var, expr in solution.items():
#         print(f"{var} = {expr}")


for solution in solutions_eq1_eq2:
        for var, expr in solution.items():
            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)

known_values

{P_d: 10,
 N_p: 20,
 n_p: 1750,
 n_g_guess: 580,
 power: 10,
 N_g_guess: 1750/29,
 VR_guess: 175/58}

In [508]:

if known_values[N_g_guess_sp] == sp.nan:  
    a = sp.nan
else:
    known_values[N_g_sp] = round(known_values[N_g_guess_sp], 0)
# known_values[N_g_sp]

# Solve equation eq3 with the updated known values

if all_values[n_g_guess_sp] == sp.nan:
    pass

else:

    solution_eq3 = sp.solve(eq3.subs(known_values), dict=True)

    # Update known values with the solution of eq3
    for sol in solution_eq3:
        for var, expr in sol.items():
            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)

known_values




{P_d: 10,
 N_p: 20,
 n_p: 1750,
 n_g_guess: 580,
 power: 10,
 N_g_guess: 1750/29,
 VR_guess: 175/58,
 N_g: 60.0000000000000,
 VR: 3.00000000000000}

#### Solve the equations multiple times bc some of them depend on others

In [509]:
# Iteratively solve equations until all variables are found

run_count = 0
while True:

    previous_known_values = known_values.copy()
    
    # Solve equations with current known values
    solutions = sp.solve([eq4, eq5.subs(known_values), eq6.subs(known_values), eq7.subs(known_values), eq8.subs(known_values), eq9.subs(known_values)], dict=True)

    # Update known values with solutions
    print(len(solutions))
    for solution in solutions:

        for var, expr in solution.items():

            if var not in known_values or known_values[var] is sp.nan:
                known_values[var] = expr.subs(known_values)    

    # Break if no new values are found
    if known_values == previous_known_values:
        break
    else:
        run_count += 1
        print(f"Run interation: {run_count}")

    if run_count > 5:
        break

# Display updated solutions
for var, value in known_values.items():
    print(f"{var} = {value}")


1
Run interation: 1
0
P_d = 10
N_p = 20
n_p = 1750
n_g_guess = 580
power = 10
N_g_guess = 1750/29
VR_guess = 175/58
N_g = 60.0000000000000
VR = 3.00000000000000
D_g = 6.00000000000000
D_p = 2.00000000000000
n_g = 583.333333333333
torque = 360.000000000000
v_t = 916.297857297022


### Lists each variable and its equation 

## List all the values

In [510]:
# assert(len(known_values) == len(all_values))
len(known_values), len(all_values)
# known_values

for var, value in known_values.items():
    all_values[var] = value


P_d = all_values[P_d_sp]
D_p = all_values[D_p_sp]
D_g = all_values[D_g_sp]
N_p = all_values[N_p_sp]
N_g = all_values[N_g_sp]
n_p = all_values[n_p_sp]
n_g = all_values[n_g_sp]
n_g_guess = all_values[n_g_guess_sp]
VR = all_values[VR_sp]
VR_guess = all_values[VR_guess_sp]
v_t = all_values[v_t_sp]
power = all_values[power_sp]
Torque = all_values[Torque_sp]

P_d = float(P_d)
D_p = float(D_p)
D_g = float(D_g)
N_p = float(N_p)
N_g = float(N_g)
n_p = float(n_p)
n_g = float(n_g)
n_g_guess = float(n_g_guess)
VR = float(VR)
VR_guess = float(VR_guess)
vt = float(v_t)
power = float(power)
Torque = float(Torque)

all_values



{P_d: 10,
 D_p: 2.00000000000000,
 D_g: 6.00000000000000,
 N_p: 20,
 N_g: 60.0000000000000,
 N_g_guess: 1750/29,
 n_p: 1750,
 n_g: 583.333333333333,
 n_g_guess: 580,
 VR: 3.00000000000000,
 VR_guess: 175/58,
 v_t: 916.297857297022,
 torque: 360.000000000000,
 power: 10}

##  Calculations for remaining values

### Find Cone angles

In [511]:
gamma = 180 / np.pi * np.arctan(N_p/N_g)

#gear cone angle

Gamma = 180 / np.pi * np.arctan(N_g/N_p)

assert(gamma + Gamma - 90 < 10**-8)


# Display the equations in LaTeX format
display(Math(latex(sp.Eq(sp.Symbol('gamma'), gamma))))
display(Math(latex(sp.Eq(sp.Symbol('Gamma'), Gamma))))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Find Cone Distances

In [512]:
#outer cone distance

A_oG = D_g / (2 * np.sin(Gamma * np.pi / 180))

A_oP = D_p / (2 * np.sin(gamma * np.pi / 180))

A_oG, A_oP
display(Math(latex(sp.Eq(sp.Symbol('A_oG'), A_oG))))
display(Math(latex(sp.Eq(sp.Symbol('A_oP'), A_oP))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Find r_m and R_m 

In [513]:
r_m = D_p / 2 - F_p / 2 * np.sin(gamma * np.pi / 180)

R_m = D_g / 2 - F_g / 2 * np.sin(Gamma * np.pi / 180)

r_m, R_m

# Display the equations in LaTeX format
display(Math(latex(sp.Eq(sp.Symbol('r_m'), r_m))))
display(Math(latex(sp.Eq(sp.Symbol('R_m'), R_m))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Find Forces

In [514]:
# assert(False) # this needs to be fixed back
# W_t = Torque / r_m

W_t = Torque / D_p * 2

wt2 = 33000 * power / v_t

# Display the equation in LaTeX format with units of lbs
display(Math(latex(sp.Eq(W_t_sp, W_t)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

Radial Force

In [515]:
W_rp = W_t * np.tan(phi * np.pi / 180) * np.cos(gamma * np.pi / 180)
# Display the equation in LaTeX format with units of lbs

display(Math(latex(sp.Eq(sp.Symbol('W_rp'), W_rp)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

Axial Force

In [516]:
W_xp = W_t * np.tan(phi * np.pi / 180) * np.sin(gamma * np.pi / 180)
display(Math(latex(sp.Eq(sp.Symbol('W_xp'), W_xp)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

In [517]:
W_rG = W_t * np.tan(phi * np.pi / 180) * np.cos(Gamma * np.pi / 180)
# Display the equation in LaTeX format with units of lbs

display(Math(latex(sp.Eq(sp.Symbol('W_rg'), W_rG)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

In [518]:
W_xg = W_t * np.tan(phi * np.pi / 180) * np.sin(Gamma * np.pi / 180)
display(Math(latex(sp.Eq(sp.Symbol('W_xg'), W_xg)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

### Find Face Width

In [519]:
if F_p is not None:
    pass

else:

    F_p = 0.3 * A_oP #nominal face width
    F_g = 0.3 * A_oG #nominal face width

    if F_p > 10 / P_d:
        F_p = 10 / P_d
        user_in = input("face width at max")

    if F_g > 10 / P_d:
        F_g = 10 / P_d
        user_in = input("face width at max")


F_p, F_g
display(Math(latex(sp.Eq(sp.Symbol('F_p'), F_p))))
display(Math(latex(sp.Eq(sp.Symbol('F_g'), F_g))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Find

### Calculate Center Distance

In [520]:
C = (N_p + N_g) / (2 * P_d)
C
display(Math(latex(sp.Eq(C_sp, C))))

<IPython.core.display.Math object>

## Calulating correction Factors

### Calculate $C_{pf}$

only does _p not _g

In [521]:
#C_pf
C_pf = None

if F_p / D_p < 0.5:
    
    #___________Use the graph too find C_pf_____________________
    C_pf = 0

elif F_p <= 1.0:
    C_pf = F_p / (10 * D_p) - 0.025
elif F_p <= 15:
    C_pf = F_p / (10 * D_p) - 0.0375 + 0.0125 * F_p

assert(C_pf is not None)

C_pf
display(Math(latex(C_pf)))

<IPython.core.display.Math object>

### Calculate $C_{ma}$

In [522]:
#C_ma

assert(C_ma_cond is not None)

C_ma = -1#None



if C_ma_cond.upper() == "commercial enclosed".upper():
    print("here")
    C_ma = 0.127 + 0.0158 * F_p - 1.093 * 10**-4 * F_p**2
elif C_ma_cond.upper() == "precision".upper():
    C_ma = 0.0675 + 0.0128 * F_p - 0.926 * 10**-4 * F_p**2
elif C_ma_cond.upper() == "extra".upper():
    C_ma = 0.0380 + 0.0102 * F_p - 0.822 * 10**-4 * F_p**2

assert(C_ma is not None)

# Display the equation in LaTeX format
display(Math(latex(sp.Eq(sp.Symbol('C_ma'), C_ma))))

here


<IPython.core.display.Math object>

In [523]:
k_m = 1 + C_ma + C_pf

display(Math(latex(sp.Eq(sp.Symbol('k_m'), k_m))))

<IPython.core.display.Math object>

k_mb

In [524]:
#k_mb
assert(k_ma_cond is not None)

k_ma = None

if k_ma_cond.upper() == "2 straddle".upper():
    k_mb = 1
elif k_ma_cond.upper() == "1 straddle".upper():
    k_mb = 1.1
elif k_ma_cond.upper() == "0 straddle".upper():
    k_mb = 1.25

assert(k_mb is not None)

# Display the equation in LaTeX format
display(Math(latex(sp.Eq(sp.Symbol('k_mb'), k_mb))))

<IPython.core.display.Math object>

### Loading Cycles $N_{cp}, N_{cg}$

In [525]:
# Number of cycles

N_cp = 60 * lifetime * n_p
N_cg = 60 * lifetime * n_g

# Print in LaTeX with scientific notation
display(Math(r"N_{cp} = " + latex(N_cp) + r" \approx " + f"{N_cp:.2e}"))
display(Math(r"N_{cg} = " + latex(N_cg) + r" \approx " + f"{N_cg:.2e}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### K_lp - need a more accurate measure for this

In [526]:
k_Lp = None

if N_cp <= 5 * 10**6:
    k_Lp = 6.514 * N_cp**(-0.1192)
elif N_cp > 5 * 10**6:
    k_Lp = 1.3558 * N_cp**(-0.0178)


assert(k_Lp is not None)
display(Math(latex(sp.Eq(sp.Symbol('k_Lp'), k_Lp))))

<IPython.core.display.Math object>

C_s

In [527]:
#idk what this is
C_s = 0.125 * F_p + 0.4375
display(Math(latex(sp.Eq(sp.Symbol('C_s'), C_s))))

<IPython.core.display.Math object>

C_xc

In [528]:
C_xc = None

if teeth == "crowned":
    C_xc = 1.5
elif teeth == "uncrowned":
    C_xc = 2
else:
    assert(False)

assert(C_xc is not None)

C_xc
display(Math(latex(sp.Eq(sp.Symbol('C_xc'), C_xc))))

<IPython.core.display.Math object>

C_lp

In [529]:
C_Lp = None#1.0 # from spme random graph

if N_cp <= 10**4:
    C_Lp = 2.0
elif N_cp > 10**4:
    C_Lp = 3.4822 * N_cp**(-0.0602)

assert(C_Lp is not None)
C_Lp

1.117834274419473

C_lg

In [530]:
C_Lg = None#1.0 # from spme random graph

if N_cg <= 10**4:
    C_Lg = 2.0
elif N_cg > 10**4:
    C_Lg = 3.4822 * N_cg**(-0.0602)

assert(C_Lg is not None)
C_Lg

1.194263395097452

### *** Implement $Y_{xx}$  and $Z_{xx}$

Put all the graphs here

## Stresses and allowable Stresses

### $S_{tp}$

In [531]:
s_t = (W_t * P_d) / (F_p * J_p) * k_o * k_s * k_m * k_v
s_t

# Display the equation in LaTeX format with units of lbs
display(Math(latex(sp.Eq(sp.Symbol('s_tp'), s_t)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

### $S_{atp}$, $S_{atG}$

In [532]:
s_atp = s_t * safety_factor * k_R / k_Lp

display(Math(latex(sp.Eq(sp.Symbol('s_atP >'), s_atp)) + r" \text{ psi}"))
# s_atp

<IPython.core.display.Math object>

### Calc $S_c$

In [539]:
S_c = C_p * np.sqrt( float( (W_t * k_o * k_m * k_v * C_s * C_xc) / (F_p * D_p * I) ) ) #

# Display the equation in LaTeX format with units of lbs
display(Math(latex(sp.Eq(sp.Symbol('S_c'), S_c)) + r" \text{ ksi}"))

<IPython.core.display.Math object>

### FInd adjusted values of $S_c$

In [534]:
S_acp = S_c * (safety_factor * C_R) / C_Lp

# Display the equation in LaTeX format with units of lbs
display(Math(latex(sp.Eq(sp.Symbol('S_acP >'), S_acp)) + r" \text{ lbs}"))

<IPython.core.display.Math object>

## Brinell Hardness Calculations

In [535]:
# Prompt the user for input
HB_grade = input("HB grade 1 or 2? ")

Contact_HB = 0
Bending_HB = 0

if int(HB_grade) == 1:
    Contact_HB = ( S_acp /1000 - 23.62 ) / 0.341
    Bending_HB = ( s_atp / 1000 - 2.1 ) / 0.044 

elif int(HB_grade) == 2:
    Contact_HB = ( S_acp / 1000 - 29.56 ) / 0.3636
    Bending_HB = ( s_atp / 1000 - 5.980 ) / 0.048

else:
    assert(False)


print(f"Contact_HB: {Contact_HB:.4f} \nBending_HB: {Bending_HB:.4f} \n")


Contact_HB: 296.5588 
Bending_HB: 298.3745 



### use tables to find the appropriate Material, and enter its brinell hardness

In [536]:
HB = 400
if HB is None:
    raise Exception("Need to enter HB value")


### Calculate actaul allowed bending and contact stresses

In [537]:
#too lazy to implement rn

S_acp = None
S_atp = None

S_acg = None
S_atg = None


if int(HB_grade) == 1:
    S_acp2 = (HB * 0.341 + 23.62 ) * 1000 
    S_atp2 = (HB * 0.044 + 2.1 ) * 1000

elif int(HB_grade) == 2:

    S_acp2 = (HB * 0.3636 + 29.56 ) * 1000
    S_atp2 = (HB * 0.048 + 5.980 ) * 1000
    
else:
    assert(False)


### Calculate Safety factor

In [541]:
contact_SF_p = S_acp2 * Y_np / (S_c * k_R)
bending_SF_p = 0#S_atp2 * Y_np / (s_t * k_R)

print(f"Contact Safety Factor (Pinion): {contact_SF_p:f} \nBending Safety Factor (Pinion): {bending_SF_p:f}\n")


print(S_acp, Y_np, S_c, k_R)


TypeError: unsupported format string passed to NaN.__format__

## Power transmitting Capacity

In [324]:
# Power transmitting capacity

P_cap = n_p * F_p * I / (126000 * k_o * k_s * k_m * k_v) * ( (S_acp2 * D_p * Z_np) / (safety_factor * k_R * C_p) ) ** 2

print(f"P_cap = {P_cap:.4f} HP")

P_cap = 9.9048 HP
